In [34]:
from FlagEmbedding import FlagModel
sentences_1 = ["样例数据-1", "样例数据-2"]
sentences_2 = ["样例数据-3", "样例数据-4"]
model = FlagModel('BAAI/bge-large-zh-v1.5', 
                  query_instruction_for_retrieval="Generate a representation for this sentence for retrieving related articles:",
                  use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

In [35]:
embeddings_1 = model.encode(sentences_1)
embeddings_2 = model.encode(sentences_2)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [36]:
followed_papers = ['Novel biomarkers in breast cancer progression',
'Applications of Microfluidics and Organ-on-a-Chip in Cancer Research',
'Applications of single-cell sequencing in cancer research: progress and perspectives',
'Artificial intelligence in cancer research, diagnosis and therapy',
'Artificial Intelligence in Cancer Research: Trends, Challenges and Future Directions',
'Artificial intelligence in histopathology: enhancing cancer research and clinical oncology',
]

other_papers = [
    'Artificial Intelligence in Cancer Research and Precision Medicine.',
    'Establishing guidelines to harmonize tumor mutational burden (TMB): in silico assessment of variation in TMB quantification across diagnostic platforms: phase I of the Friends of Cancer Research TMB Harmonization Project',
'Global cancer statistics 2018: GLOBOCAN estimates of incidence and mortality worldwide for 36 cancers in 185 countries',
'Global Cancer Statistics 2020: GLOBOCAN Estimates of Incidence and Mortality Worldwide for 36 Cancers in 185 Countries',
'Global cancer statistics 2022: GLOBOCAN estimates of incidence and mortality worldwide for 36 cancers in 185 countries',
'Journal of Cancer Research and Clinical Oncology',
'Metabolomics in cancer research and emerging applications in clinical oncology',
'Organoids in cancer research',
'Big data in basic and translational cancer research',
'Priorities for cancer research in low- and middle-income countries: a global perspective',
'Recent Advances in Thyroid Cancer Research',
'RNA sequencing: new technologies and applications in cancer research',
'Scaffold-based 3D cell culture models in cancer research',
'Spatial omics: Navigating to the golden era of cancer research',
'Statistical methods in cancer research. Volume II--The design and analysis of cohort studies.',
'UK Biobank: a globally important resource for cancer research']


In [37]:
embeddings_1 = model.encode(followed_papers)
embeddings_2 = model.encode(other_papers)

In [52]:
similarity = embeddings_1 @ embeddings_2.T
print(similarity.shape)

(6, 16)


In [ ]:
import numpy as np

In [49]:
sim_arr = np.array(similarity)
papers_sorted = sorted(list(zip(np.sum(sim_arr, axis = 0).tolist(), other_papers)),reverse=True)
papers = [p[1] for p in papers_sorted]

In [50]:
print(papers_sorted)

[(4.28125, 'Artificial Intelligence in Cancer Research and Precision Medicine.'), (3.96875, 'Journal of Cancer Research and Clinical Oncology'), (3.962890625, 'Metabolomics in cancer research and emerging applications in clinical oncology'), (3.94921875, 'RNA sequencing: new technologies and applications in cancer research'), (3.939453125, 'Scaffold-based 3D cell culture models in cancer research'), (3.8984375, 'Statistical methods in cancer research. Volume II--The design and analysis of cohort studies.'), (3.78515625, 'Big data in basic and translational cancer research'), (3.744140625, 'Spatial omics: Navigating to the golden era of cancer research'), (3.728515625, 'Priorities for cancer research in low- and middle-income countries: a global perspective'), (3.654296875, 'Organoids in cancer research'), (3.578125, 'Recent Advances in Thyroid Cancer Research'), (3.396484375, 'UK Biobank: a globally important resource for cancer research'), (3.35546875, 'Establishing guidelines to harm

In [51]:
print(papers)

['Artificial Intelligence in Cancer Research and Precision Medicine.', 'Journal of Cancer Research and Clinical Oncology', 'Metabolomics in cancer research and emerging applications in clinical oncology', 'RNA sequencing: new technologies and applications in cancer research', 'Scaffold-based 3D cell culture models in cancer research', 'Statistical methods in cancer research. Volume II--The design and analysis of cohort studies.', 'Big data in basic and translational cancer research', 'Spatial omics: Navigating to the golden era of cancer research', 'Priorities for cancer research in low- and middle-income countries: a global perspective', 'Organoids in cancer research', 'Recent Advances in Thyroid Cancer Research', 'UK Biobank: a globally important resource for cancer research', 'Establishing guidelines to harmonize tumor mutational burden (TMB): in silico assessment of variation in TMB quantification across diagnostic platforms: phase I of the Friends of Cancer Research TMB Harmonizat

## Code to implement algorithm into the website -- will add soon 

import numpy as np
from FlagEmbedding import FlagModel
model = FlagModel('BAAI/bge-large-zh-v1.5', 
                  query_instruction_for_retrieval="Generate a representation for this sentence for retrieving related articles:",
                  use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

def get_recent_papers(session, person_id, limit=100):
    papers = (
        session.query(Papers)
        .order_by(Papers.publication_date.desc())
        .limit(limit)
        .all()
    )

    paper_ids = [p.paper_id for p in papers]
    paper_titles = [p.title for p in papers]
    
    starred_titles = list(
        r[0] for r in session.query(StarredPapers.paper_id)
        .filter(StarredPapers.person_id == person_id)
        .filter(StarredPapers.paper_id.in_(paper_ids))
        .all()
    )

    print(paper_titles)
    print(starred_titles)

    embeddings_1 = model.encode(starred_titles)
    embeddings_2 = model.encode(paper_titles)

    similarity = embeddings_1 @ embeddings_2.T
    print(similarity.shape)

    sim_arr = np.array(similarity)
    papers_sorted = sorted(list(zip(np.sum(sim_arr, axis = 0).tolist(), papers)),reverse=True, key= lambda x: x[0])
    papers = [p[1] for p in papers_sorted]
